# Workshop - Human in the Loop for SageMaker Models - Module 1

In this firt module you are going to train and deploy an object detection model utilizing [Object Detection](https://docs.aws.amazon.com/sagemaker/latest/dg/object-detection.html) built-in algorithm.

If it is your first time running a notebook, check out the menus to get a feel for it. To execute the cell click into the cell and click run icon in the toolbar above or press `Shift + Enter`

## Setup
This notebook is developed and tested in a SageMaker Notebook Instance with a `ml.t3.medium` instance with SageMaker Python SDK v2. It is recommended to execute the notebook in the same environment for best experience.

In [ ]:
!pip install opencv-python-headless

In [ ]:
import os
import time
import json
import random
import cv2
import boto3
import botocore
import sagemaker
from sagemaker import get_execution_role
from sagemaker import image_uris
import matplotlib.pyplot as plt
import matplotlib.patches as patches    
import matplotlib.image as mpimg

In [ ]:
role = get_execution_role()
sess = sagemaker.Session()
bucket = sess.default_bucket()
prefix = "DEMO-ObjectDetection"

## Load Dataset and Prepare Data

_Can take up ~5-10 minutes_

In [ ]:
#%%time

# Download the dataset
!wget -P /tmp http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
!wget -P /tmp http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar

# Extract the data.
!tar -xf /tmp/VOCtrainval_06-Nov-2007.tar && rm /tmp/VOCtrainval_06-Nov-2007.tar
!tar -xf /tmp/VOCtest_06-Nov-2007.tar && rm /tmp/VOCtest_06-Nov-2007.tar

**Convert data into RecordIO**

RecordIO is a highly efficient binary data format from MXNet that makes it easy and simple to prepare the dataset and transfer to the instance that will run the training job. To generate a RecordIO file, we will use the tools from MXNet. The provided tools will first generate a list file and then use the im2rec tool to create the RecordIO file.

In [ ]:
!python tools/prepare_dataset.py --dataset pascal --year 2007 --set trainval --target VOCdevkit/train.lst
!python tools/prepare_dataset.py --dataset pascal --year 2007 --set test --target VOCdevkit/val.lst --no-shuffle

In [ ]:
# Upload the RecordIO files to train and validation channels
train_channel = prefix + "/train"
validation_channel = prefix + "/validation"

sess.upload_data(path="VOCdevkit/train.rec", bucket=bucket, key_prefix=train_channel)
sess.upload_data(path="VOCdevkit/val.rec", bucket=bucket, key_prefix=validation_channel)

s3_train_data = "s3://{}/{}".format(bucket, train_channel)
s3_validation_data = "s3://{}/{}".format(bucket, validation_channel)
s3_output_location = "s3://{}/{}/output".format(bucket, prefix)

## Train with SageMaker Built-in Algorithm

In [ ]:
training_image = image_uris.retrieve(
    region=sess.boto_region_name, framework="object-detection", version="latest"
)

In [ ]:
od_model = sagemaker.estimator.Estimator(
    training_image,
    role,
    instance_count=1,
    instance_type="ml.p3.2xlarge",
    volume_size=50,
    max_run=360000,
    input_mode="File",
    output_path=s3_output_location,
    sagemaker_session=sess,
)

In [ ]:
od_model.set_hyperparameters(
    base_network="resnet-50",
    use_pretrained_model=1,
    num_classes=20,
    mini_batch_size=32,
    epochs=1,
    learning_rate=0.001,
    lr_scheduler_step="3,6",
    lr_scheduler_factor=0.1,
    optimizer="sgd",
    momentum=0.9,
    weight_decay=0.0005,
    overlap_threshold=0.5,
    nms_threshold=0.45,
    image_shape=300,
    label_width=350,
    num_training_samples=16551,
)

In [ ]:
train_data = sagemaker.inputs.TrainingInput(
    s3_train_data,
    distribution="FullyReplicated",
    content_type="application/x-recordio",
    s3_data_type="S3Prefix",
)
validation_data = sagemaker.inputs.TrainingInput(
    s3_validation_data,
    distribution="FullyReplicated",
    content_type="application/x-recordio",
    s3_data_type="S3Prefix",
)
data_channels = {"train": train_data, "validation": validation_data}

In [ ]:
od_model.fit(inputs=data_channels, logs=True)

## Deploy Endpoint for Inferences and test

In [ ]:
endpoint_name = 'DEMO-ObjectDetection-endpoint'

object_detector = od_model.deploy(
    initial_instance_count=1,
    endpoint_name = endpoint_name,
    instance_type="ml.m5.xlarge",
    serializer = sagemaker.serializers.IdentitySerializer('image/jpeg')
)

In [ ]:
def visualize_detection(img_file, dets, classes=[], thresh=0.6):
    """
    visualize detections in one image
    Parameters:
    ----------
    img : numpy.array
        image, in bgr format
    dets : numpy.array
        ssd detections, numpy.array([[id, score, x1, y1, x2, y2]...])
        each row is one object
    classes : tuple or list of str
        class names
    thresh : float
        score threshold
    """
    img=mpimg.imread(img_file)
    f, ax = plt.subplots(1, 1)
    ax.imshow(img)
    height = img.shape[0]
    width = img.shape[1]
    colors = dict()
    output = []
    for det in dets:
        (klass, score, x0, y0, x1, y1) = det
        cls_id = int(klass)
        class_name = str(cls_id)
        if classes and len(classes) > cls_id:
            class_name = classes[cls_id]
        output.append([class_name, score])
        if score < thresh:
            continue
        if cls_id not in colors:
            colors[cls_id] = (random.random(), random.random(), random.random())
        xmin = int(x0 * width)
        ymin = int(y0 * height)
        xmax = int(x1 * width)
        ymax = int(y1 * height)
        rect = patches.Rectangle((xmin, ymin), xmax - xmin,
                                 ymax - ymin, fill=False,
                                 edgecolor=colors[cls_id],
                                 linewidth=3.5)
        ax.add_patch(rect)


        ax.text(xmin, ymin - 2,
                '{:s} {:.3f}'.format(class_name, score),
                bbox=dict(facecolor=colors[cls_id], alpha=0.5),
                          fontsize=12, color='white')

    return f, output
    
def load_and_predict(file_name, predictor, threshold=0.5):
    """
    load an image, make object detection to an predictor, and visualize detections
    Parameters:
    ----------
    file_name : str
        image file location, in str format
    predictor : sagemaker.predictor.RealTimePredictor
        a predictor loaded from hosted endpoint
    threshold : float
        score threshold for bounding box display
    """
    with open(file_name, 'rb') as image:
        f = image.read()
        b = bytearray(f)
    results = predictor.predict(b)
    detections = json.loads(results)
    
    fig, detection_filtered = visualize_detection(file_name, detections['prediction'], 
                                                   object_categories, threshold)
    
    return results, detection_filtered, fig

In [ ]:
object_categories = ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 
                     'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 
                     'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']

In [ ]:
test_photos_index = ['980382', '276517', '1571457']

if not os.path.isdir('sample-a2i-images'):
    os.mkdir('sample-a2i-images')
    
for ind in test_photos_index:
    !curl https://images.pexels.com/photos/{ind}/pexels-photo-{ind}.jpeg > sample-a2i-images/pexels-photo-{ind}.jpeg

In [ ]:
test_photos = ['sample-a2i-images/pexels-photo-980382.jpeg', # motorcycle
               'sample-a2i-images/pexels-photo-276517.jpeg', # bicycle
               'sample-a2i-images/pexels-photo-1571457.jpeg'] # sofa

In [ ]:
results = load_and_predict(test_photos[2], object_detector, threshold=0.2)[0]
dict_results = json.loads(results.decode('utf8'))['prediction']

In [ ]:
visualize_detection(test_photos[2], dict_results, object_categories, thresh=0.4)[0]

## Clean Up (finish module 2 and 3 before)

In [ ]:
# object_detector.delete_endpoint()